## Config


In [1]:
import pandas as pd
import numpy as np

## Data

In [2]:
df = pd.read_excel('M3C.xls', sheet_name='M3Year')
df.head()

,Series,N,NF,Category,Starting Year,Unnamed: 5,1,2,3,4,...,38,39,40,41,42,43,44,45,46,47
0,N 1,20,6,MICRO,1975,1,940.66,1084.86,1244.98,1445.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N 2,20,6,MICRO,1975,1,1991.05,2306.40,2604.00,2992.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N 3,20,6,MICRO,1975,1,1461.57,1692.50,2193.82,2459.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N 4,20,6,MICRO,1975,1,744.54,1105.16,1417.40,1838.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N 5,20,6,MICRO,1975,1,4977.18,5248.00,5370.00,6184.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.drop(columns = 'Unnamed: 5', inplace = True)
df['Series'] = df['Series'].str.replace('N ', '').astype(int)

## Melt from wide format to long format

In [4]:
# Using pandas.melt to transform the DataFrame
melted_df = pd.melt(df, id_vars=['Series', 'N', 'NF', 'Category', 'Starting Year'], 
                    var_name='Measurement', value_name='Value')

print(melted_df.head())


   Series   N  NF      Category  Starting Year Measurement    Value
0       1  20   6  MICRO                  1975           1   940.66
1       2  20   6  MICRO                  1975           1  1991.05
2       3  20   6  MICRO                  1975           1  1461.57
3       4  20   6  MICRO                  1975           1   744.54
4       5  20   6  MICRO                  1975           1  4977.18


In [5]:
melted_df = melted_df.sort_values(by=['Series', 'Category'])
melted_df

,Series,N,NF,Category,Starting Year,Measurement,Value
0,1,20,6,MICRO,1975,1,940.66
645,1,20,6,MICRO,1975,2,1084.86
1290,1,20,6,MICRO,1975,3,1244.98
1935,1,20,6,MICRO,1975,4,1445.02
2580,1,20,6,MICRO,1975,5,1683.17
...,...,...,...,...,...,...,...
27734,645,38,6,OTHER,1955,43,NaN
28379,645,38,6,OTHER,1955,44,NaN
29024,645,38,6,OTHER,1955,45,NaN
29669,645,38,6,OTHER,1955,46,NaN


In [6]:
melted_df.reset_index(drop=True, inplace=True)

In [7]:
melted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30315 entries, 0 to 30314
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series         30315 non-null  int64  
 1   N              30315 non-null  int64  
 2   NF             30315 non-null  int64  
 3   Category       30315 non-null  object 
 4   Starting Year  30315 non-null  int64  
 5   Measurement    30315 non-null  object 
 6   Value          18319 non-null  float64
dtypes: float64(1), int64(4), object(2)
memory usage: 1.6+ MB


## Add a datetime column

In [8]:
melted_df['Measurement'] = melted_df['Measurement'].astype(int)

# Calculate the actual year by adding the 'Measurement' offset to the 'Starting Year' and subtracting 1
# Then, create the datetime column representing January 1st of the calculated year
melted_df['Date'] = pd.to_datetime((melted_df['Starting Year'] + melted_df['Measurement'] - 1).astype(str) + '-01-01')

melted_df.head()

,Series,N,NF,Category,Starting Year,Measurement,Value,Date
0,1,20,6,MICRO,1975,1,940.66,1975-01-01
1,1,20,6,MICRO,1975,2,1084.86,1976-01-01
2,1,20,6,MICRO,1975,3,1244.98,1977-01-01
3,1,20,6,MICRO,1975,4,1445.02,1978-01-01
4,1,20,6,MICRO,1975,5,1683.17,1979-01-01


In [9]:
melted_df.drop(columns = ['Starting Year','Measurement'], inplace = True)

In [21]:
melted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30315 entries, 0 to 30314
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Series    30315 non-null  int64         
 1   N         30315 non-null  int64         
 2   NF        30315 non-null  int64         
 3   Category  30315 non-null  object        
 4   Value     18319 non-null  float64       
 5   Date      30315 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 1.4+ MB


In [22]:
melted_df.to_parquet('M3_yearly_processed.parquet', index=False, compression='snappy', engine='pyarrow')